In [1]:
from KeRLas import Brain, TimedGymEnv, Trainer
from KeRLas.models import defaultQModel
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel, DQN
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from cartpole import CartPoleEnv

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(CartPoleEnv(), tlimit=300)

space = env.observation_space
high = np.array(
    [
        2.4,
        1.0,
        12 * 2 * math.pi / 360,
        1.0
    ]
)
env.RandomObservationSpace = gym.spaces.Box(-high, high)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n


#for i in range(10):
#    next(brain.Memory.generate_samples(20))
#_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [6]:
rlmodel = DirectDiffModel(defaultQModel(observation_width, nactions), 0.9)
policy = BoltzmannQPolicy(0.0001)
brain = Brain(rlmodel, policy, training_policies=map(BoltzmannQPolicy, [10.0, 1.0, 0.1, 0.01, 0.001]))


In [7]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record, info):
        env.render()

trainer = Trainer(env, brain, 0.00, 10000)   

from KeRLas import GymPlayer
p = GymPlayer(env, brain, callback=Callback())

#temps = [10.0, 1.0, 0.1, 0.01, 0.001]

mbsize = 50

for i in xrange(20):
    metrics = trainer.train(mbsize, 1000, 10)
    
    nsum = 0
    N = 5
    with brain.training(False):
        for _ in range(N):
            history = p.runEpisode()
            #print len(history)
            nsum += len(history)
        print float(nsum)/N, metrics

36.8 0.08482725
9.8 0.050808106
22.6 0.026157703
47.4 0.050469443
69.0 0.048904303
90.8 0.088459276
244.4 0.06096409
300.0 0.09482729
256.0 0.10696578
300.0 0.09076548
300.0 0.13666454
291.2 0.09179892
255.8 0.11217169
240.8 0.07728295
233.4 0.06326427
261.6 0.12663162
191.4 0.06528723
228.6 0.032341875
156.0 0.06320083
230.6 0.04938815
